In [ ]:
import datetime
import os
from glob import glob

import numpy as np
import torch
import torch.nn.functional as F

from omegaconf import OmegaConf
from torchinfo import summary
from app.config import root_dir
from app.metrics import BYUFbeta
from app.metrics.metrics import get_topk_by_id, thresholder, filter_negatives
from app.models.lightning import Net
from app.utils import get_data, get_data_loader
from app.processings.post_processing import get_output_size, reconstruct, simple_nms

In [ ]:
from numpy.typing import NDArray
from scipy.spatial import KDTree
from torchmetrics.utilities import dim_zero_cat

In [ ]:
OmegaConf.register_new_resolver("root_dir", resolver=root_dir, replace=True)
OmegaConf.register_new_resolver("eval", resolver=eval, replace=True)

In [ ]:
os.environ["ISTPUVM"] = "1"
os.environ["PJRT_DEVICE"] = "CPU"
os.environ["PT_XLA_DEBUG_LEVEL"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TPU_ACCELERATOR_TYPE"] = "v3-8"
os.environ["TPU_CHIPS_PER_HOST_BOUNDS"] = "2,2,1"
os.environ["TPU_HOST_BOUNDS"] = "1,1,1"
os.environ["TPU_RUNTIME_METRICS_PORTS"] = "8431,8432,8433,8434"
os.environ["TPU_SKIP_MDS_QUERY"] = "1"
os.environ["TPU_WORKER_HOSTNAMES"] = "localhost"
os.environ["TPU_WORKER_ID"] = "0"
os.environ["XLA_TENSOR_ALLOCATOR_MAXSIZE"] = "100000000"

In [ ]:
cfg = OmegaConf.load("../src/app/config/config.yaml")

In [ ]:
cfg.max_th = 0.7

In [ ]:
train_df, val_df = get_data(cfg, mode="fit")
train_loader = get_data_loader(cfg, train_df, mode="train")
val_loader = get_data_loader(cfg, val_df, mode="validation")

In [ ]:
metric = BYUFbeta(cfg, compute_on_cpu=True, dist_sync_on_step=True)

In [ ]:
zyxic = torch.load("")

In [ ]:
targets = torch.from_numpy(
    val_df[val_df.id.isin(np.unique(zyxic[:, 3]))][["z", "y", "x", "id", "vxs"]]
    .copy()
    .values
)

In [ ]:
results = metric(zyxic, targets)

In [ ]:
results

In [ ]:
model = Net(cfg)

In [ ]:
batch = next(iter(val_loader))

In [ ]:
model.eval()
output = model(batch)